In [2]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/kunjee17/mahabharata/master/txt_data/volume1.txt

--2024-07-18 23:42:06--  https://raw.githubusercontent.com/kunjee17/mahabharata/master/txt_data/volume1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3693939 (3.5M) [text/plain]
Saving to: ‘volume1.txt’

volume1.txt         100%[===================>]   3.52M  --.-KB/s    in 0.05s   

2024-07-18 23:42:06 (66.5 MB/s) - ‘volume1.txt’ saved [3693939/3693939]



In [3]:
# Step 1: Read the text from the file
with open('volume1.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Initialize variables
word_count = 0
index = 0

# Step 2: Split the text into words while counting them
while word_count < 2075 and index < len(text):
    if text[index].isspace():
        if index > 0 and not text[index - 1].isspace():
            word_count += 1
    index += 1

# Step 3: Skip the first 2075 words
filtered_text = text[index:]

# Print the length of the filtered dataset
print("length of filtered dataset in characters: ", len(filtered_text))


length of filtered dataset in characters:  3681539


In [4]:
# let's look at the first 1000 characters
text = filtered_text
print(text[:1000])


ADI PARVA

SECTION I

Om! Having bowed down to Narayana and Nara, the most exalted male being,
and also to the goddess Saraswati, must the word Jaya be uttered.

Ugrasrava, the son of Lomaharshana, surnamed Sauti, well-versed in the
Puranas, bending with humility, one day approached the great sages of
rigid vows, sitting at their ease, who had attended the twelve years'
sacrifice of Saunaka, surnamed Kulapati, in the forest of Naimisha. Those
ascetics, wishing to hear his wonderful narrations, presently began to
address him who had thus arrived at that recluse abode of the inhabitants
of the forest of Naimisha. Having been entertained with due respect by
those holy men, he saluted those Munis (sages) with joined palms, even
all of them, and inquired about the progress of their asceticism. Then
all the ascetics being again seated, the son of Lomaharshana humbly
occupied the seat that was assigned to him. Seeing that he was
comfortably seated, and recovered from fatigue, one of the Rish

In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"$%&'()*,-./0123456789:;?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz
83


In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[64, 65, 65, 1, 76, 64, 61, 74, 61]
hii there


In [7]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([3681539]) torch.int64
tensor([ 0, 29, 32, 37,  1, 44, 29, 46, 50, 29,  0,  0, 47, 33, 31, 48, 37, 43,
        42,  1, 37,  0,  0, 43, 69,  2,  1, 36, 57, 78, 65, 70, 63,  1, 58, 71,
        79, 61, 60,  1, 60, 71, 79, 70,  1, 76, 71,  1, 42, 57, 74, 57, 81, 57,
        70, 57,  1, 57, 70, 60,  1, 42, 57, 74, 57, 11,  1, 76, 64, 61,  1, 69,
        71, 75, 76,  1, 61, 80, 57, 68, 76, 61, 60,  1, 69, 57, 68, 61,  1, 58,
        61, 65, 70, 63, 11,  0, 57, 70, 60,  1, 57, 68, 75, 71,  1, 76, 71,  1,
        76, 64, 61,  1, 63, 71, 60, 60, 61, 75, 75,  1, 47, 57, 74, 57, 75, 79,
        57, 76, 65, 11,  1, 69, 77, 75, 76,  1, 76, 64, 61,  1, 79, 71, 74, 60,
         1, 38, 57, 81, 57,  1, 58, 61,  1, 77, 76, 76, 61, 74, 61, 60, 13,  0,
         0, 49, 63, 74, 57, 75, 74, 57, 78, 57, 11,  1, 76, 64, 61,  1, 75, 71,
        70,  1, 71, 62,  1, 40, 71, 69, 57, 64, 57, 74, 75, 64, 57, 70, 57, 11,
         1, 75, 77, 74, 70, 57, 69, 61, 60,  1, 47, 57, 77, 76, 65, 11,  1, 79,
      

In [8]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([ 0, 29, 32, 37,  1, 44, 29, 46, 50])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([0]) the target: 29
when input is tensor([ 0, 29]) the target: 32
when input is tensor([ 0, 29, 32]) the target: 37
when input is tensor([ 0, 29, 32, 37]) the target: 1
when input is tensor([ 0, 29, 32, 37,  1]) the target: 44
when input is tensor([ 0, 29, 32, 37,  1, 44]) the target: 29
when input is tensor([ 0, 29, 32, 37,  1, 44, 29]) the target: 46
when input is tensor([ 0, 29, 32, 37,  1, 44, 29, 46]) the target: 50


In [11]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[64, 61, 74, 60, 75,  1, 71, 62],
        [75, 71, 11,  1, 43,  1, 32, 74],
        [ 1, 75, 76, 57, 76, 61, 13,  1],
        [64, 76, 81,  1, 57, 74, 69, 75]])
targets:
torch.Size([4, 8])
tensor([[61, 74, 60, 75,  1, 71, 62,  1],
        [71, 11,  1, 43,  1, 32, 74, 57],
        [75, 76, 57, 76, 61, 13,  1, 29],
        [76, 81,  1, 57, 74, 69, 75, 11]])
----
when input is [64] the target: 61
when input is [64, 61] the target: 74
when input is [64, 61, 74] the target: 60
when input is [64, 61, 74, 60] the target: 75
when input is [64, 61, 74, 60, 75] the target: 1
when input is [64, 61, 74, 60, 75, 1] the target: 71
when input is [64, 61, 74, 60, 75, 1, 71] the target: 62
when input is [64, 61, 74, 60, 75, 1, 71, 62] the target: 1
when input is [75] the target: 71
when input is [75, 71] the target: 11
when input is [75, 71, 11] the target: 1
when input is [75, 71, 11, 1] the target: 43
when input is [75, 71, 11, 1, 43] the target: 1
when input is [75

In [12]:
print(xb) # our input to the transformer

tensor([[64, 61, 74, 60, 75,  1, 71, 62],
        [75, 71, 11,  1, 43,  1, 32, 74],
        [ 1, 75, 76, 57, 76, 61, 13,  1],
        [64, 76, 81,  1, 57, 74, 69, 75]])


our tesnor is BxTxC

loss should be in the range of -ln(1/83) -> 4.418

In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    # each token directly reads off the logits for the next token from the lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):

    # idx and targets are both (B, T) tensors of integers
    logits = self.token_embedding_table(idx) # (B,T,C) batch->4 by time->8 by channel->83(vocab size)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets  = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context take BxT and make it BxT+1, +2 and so on as many max token
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 83])
tensor(4.8918, grad_fn=<NllLossBackward0>)

0071vG'J:&!'.R*%&3[Y ;dqVZ"cM!?mkZ,jPBwd:1%7LRyI
Bth9aao"NlB]c0rA(C5j6P]zc]?E0N--:o!2Peigebmp7av%&FK


In [14]:
# create a PyTorch optimizer AdamW optimizer of torch
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [15]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())


2.540411949157715


In [16]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=1000)[0].tolist()))



d, ard alsethe brst
oshugas Janow-' roath es withe cud irilararby ECLo he q, a,
"Kuo s the, ale ththy ty terot ason sper amy, Patembe. r uongalaHe oWhentht Pang g, tofomaurf jue s. Sjomgrd bepr tren bour bi ounghe, ben  O henve aco SE, bert ay, a
Hatarank4Maurredraryar t andito tht mourerjuavay indiy iverd uted the,
Than areve "
ldrlimofounthfrorere cen our, hen p) as3Ksa'Sr anu t tha be
mof mis h crer fofofede Thetibe ma tiee ameng orecof end o borasTas authisthake skewinfred wn L-ato, bel th th;
ZVesen o tor itheren, crisaby as onsacencurisokere tawiJa, aveay
tothand ctr O tononofofrot wivath cld te-Srevede thes.' I g, wit Ma t lucty wed d, PuvirAnghair
ticof m)


wiet he howieranas ottheabou hop
tyame ththixCond ify bales th ily wige troriaka r thera kind aliidsseditorathangravitour bys wa'Dhighath s is Antawisone
an As, s uleddousupis, l
wadin. arenat d s a ithos atifigthes Dofungh ok woind athe
Thueghope at. theg, tri ty?"IVa, ints tssomey ts ts ve Narbofos btas. ararino Shin cin

## Trick in self attention

In [17]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [18]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [19]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [20]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [21]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [22]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

In [23]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True) ## row sum is 1
b = torch.randint(0,10,(3,2)).float()
c = a @ b ## multiplication
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [24]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [25]:
k.var()

tensor(0.9331)

In [26]:
q.var()

tensor(0.8879)

In [27]:
wei.var()

tensor(0.8150)

In [28]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [29]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [30]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/kunjee17/mahabharata/master/txt_data/volume1.txt
with open('volume1.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Initialize variables
word_count = 0
index = 0

# Step 2: Split the text into words while counting them
while word_count < 2075 and index < len(text):
    if text[index].isspace():
        if index > 0 and not text[index - 1].isspace():
            word_count += 1
    index += 1

# Step 3: Skip the first 2075 words
filtered_text = text[index:]

text = filtered_text

with open('volume1.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Initialize variables
word_count = 0
index = 0
word_positions = []

# Track the start and end positions of each word
while index < len(text):
    if text[index].isspace():
        if index > 0 and not text[index - 1].isspace():
            word_count += 1
        word_positions.append(index)
    index += 1

# Skip the first 2075 words
if word_count > 2075:
    start_index = word_positions[2075] if len(word_positions) > 2075 else len(text)
else:
    start_index = 0

# Calculate the end index to filter out the last 2947 words
end_word_count = word_count - 2947
if end_word_count > 2075:
    end_index = word_positions[end_word_count] if len(word_positions) > end_word_count else len(text)
else:
    end_index = len(text)

# Get the filtered text
filtered_text = text[start_index:end_index]

text = filtered_text
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

10.798926 M parameters
step 0: train loss 4.4813, val loss 4.4808
step 500: train loss 1.6608, val loss 1.6865
step 1000: train loss 1.2654, val loss 1.3048
step 1500: train loss 1.1463, val loss 1.1965
step 2000: train loss 1.0806, val loss 1.1441
step 2500: train loss 1.0449, val loss 1.1140
step 3000: train loss 1.0079, val loss 1.0870
step 3500: train loss 0.9819, val loss 1.0670
step 4000: train loss 0.9565, val loss 1.0532
step 4500: train loss 0.9345, val loss 1.0369
step 4999: train loss 0.9282, val loss 1.0369

Oarnas, they beheld there free from all their five household, above the entire
handsome.'"



SECTION CLXV

(Vasudeva Parva continued)


"Sauti said, 'Then the parva takil her to Kama at the solitary hermit!"



SECTION XCIII

"Yudhishthira said, 'O best of Brahmanas, it behoveth thee to listen to Ugrahista.
Tell me as all say, 'The blessed and having become happiers of Indra could
eat these words, I was a should not! Follow me, O sinless one, I have my
signs willingly 

10001